In [1]:
import pickle
pkl_sents = open('sequences_digit_VH.pkl', 'rb')
sequences = pickle.load(pkl_sents)
pkl_sents.close()

pkl_tokens = open('tokenizer_VH.pkl', 'rb')
tokenizer = pickle.load(pkl_tokens)
pkl_tokens.close()

In [2]:
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
print('Total Sequences: %d' % len(sequences))

Vocabulary Size: 20849
Total Sequences: 2926646


In [3]:
from keras.preprocessing.sequence import pad_sequences
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 3


In [5]:
from keras.models import Sequential
from keras.layers import Embedding,BatchNormalization,Dropout,LSTM,Dense
# define model

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_length-1))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation= 'softmax'))

print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 50)             1042450   
_________________________________________________________________
lstm (LSTM)                  (None, 2, 100)            60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 20849)             2105749   
Total params: 3,299,099
Trainable params: 3,299,099
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# split into input and output elements
sequences = np.array(sequences)[:200000]

X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
import keras
from keras.callbacks import EarlyStopping

# Stop training if 'val_loss' stops improving for over 10 epochs
early_stopping=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,verbose=1)

# Save the best model to .h5 file
best_model=keras.callbacks.ModelCheckpoint(filepath='best_Language_model_2.h5',monitor='val_loss',save_best_only=True,verbose=1)

# Start training
model.fit(X, y, epochs=100,validation_split=0.3,callbacks=[early_stopping,best_model],verbose=1)

In [7]:
from keras.models import load_model
import string
import re, string
from tqdm import tqdm
from pyvi import ViTokenizer
from keras.preprocessing.text import Tokenizer
model = load_model('best_Language_model_2.h5')

In [63]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    
    # generate a fixed number of words
    for i in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        
        # pre-pad sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        
        # predict probabilities for each word
        ypredict=np.argmax(model.predict(encoded) ,axis=1)
        
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == int(ypredict):
                out_word = word
                break
            # append to input
        in_text += ' ' + out_word
    return in_text

In [67]:
# evaluate model
print(generate_seq(model, tokenizer, max_length-1, 'Diễn viên hài',4))


Diễn viên hài của mình có một


In [68]:
print(generate_seq(model, tokenizer, max_length-1, 'Đến nay', 4))

Đến nay được một người mẫu
